In [3]:
print("Hello, Plagiaator!")

Hello, Plagiaator!


# Artiklite kogumine

In [4]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import os.path

def parseArticle(url):
  req=Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})
  op = urlopen(req)
  rd = op.read()
  op.close
  soup = BeautifulSoup(rd, 'lxml')
  title=soup.find("h1").text
  #print(title)
  textClasses = soup.find_all('div', class_="text flex-row")
  keywordsClass = soup.find_all('a', class_="btn small round modest")
  keywords=[]
  for el in textClasses:
    text=el.text
  for el in keywordsClass:
    #print(el)
    keywords.append(el.text)
  return title, text, keywords

if not os.path.isfile("article_data.csv"):
  #File full of links to articles from ERR.
  with open("articles.txt") as f:
    id_list=[]
    title_list=[]
    text_list=[]
    keywords_list=[]
    i=0
    for url in f:
      # print(i)
      title, text, keywords= parseArticle(url)
      id_list.append(i)
      title_list.append(title)
      text_list.append(text)
      keywords_list.append(keywords)
      i+=1
    #print(id_list)
    #print(title_list)
    #print(text_list)
    #print(keywords_list)

    #Convert to pandas dataframe.
    table=pd.DataFrame({'ID':id_list,
                      'Title':title_list,
                      'Text':text_list,
                      'Keywords':keywords_list})
    table.to_csv('article_data.csv')
    # display(table)
else:
  table = pd.read_csv('article_data.csv')

In [5]:
table.dropna(inplace=True)

# Artiklite kirjutamise õppimine

In [21]:
import nltk
from tensorflow.keras.layers import Embedding, LSTM, Dense, concatenate
from tensorflow.keras import Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [22]:
import tensorflow as tf
import numpy as np
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

WINDOW_SIZE = 50

def window_data(data, window_size):
  for i in range(0, len(data), window_size):
    window = data[i:i+window_size]
    window_padded = pad_sequences([window], maxlen=window_size, padding='post')[0]
    yield window_padded

titles = table['Title'].values
texts = table['Text'].values
keywords = table['Keywords'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

titles_sequences = tokenizer.texts_to_sequences(titles)
texts_sequences = tokenizer.texts_to_sequences(texts)
keywords_sequences = tokenizer.texts_to_sequences(keywords)

num_words = len(tokenizer.word_index) + 1

texts_windows = [list(window_data(s, WINDOW_SIZE)) for s in texts_sequences]
# texts_windows = [i for s in texts_windows for i in s]

keywords_windows = [list(window_data(s, WINDOW_SIZE)) for s in keywords_sequences]

texts_windows_padded = pad_sequences(texts_sequences, maxlen=WINDOW_SIZE, padding='post')
keywords_windows_padded = pad_sequences(keywords_sequences, maxlen=WINDOW_SIZE, padding='post')

titles_padded = pad_sequences(titles_sequences, maxlen=WINDOW_SIZE, padding='post')

In [23]:
input_keywords = Input(shape=(WINDOW_SIZE,), name='keywords')
input_title = Input(shape=(WINDOW_SIZE,), name='title')

x1 = Embedding(input_dim=num_words, output_dim=128, input_length=WINDOW_SIZE)(input_keywords)
x1 = LSTM(units=128, return_sequences=True)(x1)

x2 = Embedding(input_dim=num_words, output_dim=128, input_length=WINDOW_SIZE)(input_title)
x2 = LSTM(units=128, return_sequences=True)(x2)

x = concatenate([x1, x2])

output_title = Dense(units=num_words, activation='softmax', name='title_output')(x)
output_text = Dense(units=num_words, activation='softmax', name='paragraph_text')(x)

model = Model(inputs=[input_title, input_keywords], outputs=[output_title, output_text])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
class StopCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('loss') < 0.1:
      print('Reached 90% accuracy so cancelling training!')
      self.model.stop_training = True

In [25]:
model.fit([titles_padded, keywords_windows_padded], [titles_padded, texts_windows_padded],
  batch_size=32, epochs=500, callbacks=[StopCallback()])

Epoch 1/500
12/12 [==============================] - 5s 168ms/step - loss: 20.9236 - title_output_loss: 10.3588 - paragraph_text_loss: 10.5648 - title_output_accuracy: 0.7910 - paragraph_text_accuracy: 0.0041
Epoch 2/500
12/12 [==============================] - 2s 167ms/step - loss: 15.8314 - title_output_loss: 6.6162 - paragraph_text_loss: 9.2152 - title_output_accuracy: 0.8676 - paragraph_text_accuracy: 0.0232
Epoch 3/500
12/12 [==============================] - 2s 167ms/step - loss: 10.0337 - title_output_loss: 1.7677 - paragraph_text_loss: 8.2660 - title_output_accuracy: 0.8676 - paragraph_text_accuracy: 0.0290
Epoch 4/500
12/12 [==============================] - 2s 167ms/step - loss: 9.4435 - title_output_loss: 1.3695 - paragraph_text_loss: 8.0740 - title_output_accuracy: 0.8676 - paragraph_text_accuracy: 0.0498
Epoch 5/500
12/12 [==============================] - 2s 167ms/step - loss: 9.3015 - title_output_loss: 1.2932 - paragraph_text_loss: 8.0083 - title_output_accuracy: 0.8676

# Uute artiklite genereerimine

Algselt oli plaan genereerida mitme võtmesõna alusel, kuid millegi pärast ta ei taha enam üle ühe sõna sisendiks võtta.

In [26]:
def predict_article(keywords):
  keywords_sequences = tokenizer.texts_to_sequences(keywords)
  keywords_windows_padded = pad_sequences(keywords_sequences, maxlen=WINDOW_SIZE, padding='post')
  
  dummy_title = np.zeros((1, WINDOW_SIZE))

  [title_pred, text_pred] = model.predict([dummy_title, keywords_windows_padded])

  title_pred = np.argmax(title_pred, axis=1)
  text_pred = np.argmax(text_pred, axis=1)

  return title_pred, text_pred

def generate_article(keywords):
  title_pred, text_pred = predict_article(keywords)
  title = tokenizer.sequences_to_texts(title_pred)[0]
  text = tokenizer.sequences_to_texts(text_pred)[0]
  return title, text

In [27]:
WORDS_PER_ROW = 20

def prepare_text(text):
  words = text.split(" ")

  lines = []
  for i in range(len(words)//WORDS_PER_ROW):
    lines.append(words[i*WORDS_PER_ROW:(i+1)*WORDS_PER_ROW])

  return "\n".join(" ".join(line) for line in lines)

In [31]:
keyword = "Eesti" #@param {type: "string", title:"K"}
title, text = generate_article([keyword])

print(f"""{prepare_text(title)}
---
{prepare_text(text)}
""")

1/1 [==============================] - 0s 20ms/step
eesti mitte tuleb mitte oleks mitte mitte oleks mitte meie väga pole mitte oleks mitte oleks mitte oleks mitte mitte
oleks oleks oleks mitte meie mitte oleks mitte oleks mitte oleks mitte mitte võib oleks kui oleks oleks oleks pole
oleks oleks mitte oleks oleks oleks aastal oleks oleks pole tuleb mitte aastal ka oleks oleks mitte oleks oleks mitte
väga mitte oleks mitte mitte vene oleks seda oleks tuleb oleks oleks oleks oleks oleks mitte oleks seda pole mitte
oleks oleks oleks oleks oleks oleks väga mitte mitte mitte mida mitte mitte oleks mitte oleks nende oleks oleks oleks
väga oleks väga oleks oleks nii nende oleks tuleb mitte võib oleks mitte oleks pole oleks oleks mitte väga nende
---
ta ja ning ja ja siis ja ja siis kui ja on seda aastal on ja siis ning ja venemaa
on ja ja aasta oli ja ning juba on siis ja ja ning eesti on on on ning või venemaa
on seda juba ning on on ning siis ja aga ning ning ning ei on seda siis siis ning j